In [1]:
import torch
import os
import shutil
import numpy as np
import csv
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import torch
import pandas as pd
import lightning as L

from minerva.models.nets.time_series.gans import TTSGAN_Generator, TTSGAN_Discriminator, TTSGAN_Encoder, GAN
from GANModels import Discriminator, Generator, Encoder

from lightning.pytorch.callbacks import Callback

import random

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:53: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
experiment_number = 0
version = 0
n_epochs = 15
batch_size = 64
SAC = ['Sit', 'Stand', 'Walk', 'Upstairs', 'Downstairs', 'Run']
beta1 = 0.9
beta2 = 0.999
gen_lr = 0.0001
dis_lr = 0.0003
assimetrical_percentage = 1.0

### Testing diferences

In [3]:
random_seed = 42

def set_seed(seed: int = 42):  
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)

In [4]:
set_seed()
model_minerva = GAN(generator = TTSGAN_Generator(seq_len = 60, channels = 6),
            discriminator = TTSGAN_Discriminator(seq_len = 60, channels = 6), 
            loss_gen = torch.nn.MSELoss(),
            loss_dis = torch.nn.MSELoss(),
            assimetrical_percentage = assimetrical_percentage,
            generator_lr = gen_lr,
            discriminator_lr = dis_lr,
            beta1 = beta1,
            beta2 = beta2,
            )

In [30]:
set_seed()
model_original = GAN(generator = Generator(seq_len = 60, channels = 6),
            discriminator = Discriminator(seq_len = 60, channels = 6), 
            loss_gen = torch.nn.MSELoss(),
            loss_dis = torch.nn.MSELoss(),
            assimetrical_percentage = assimetrical_percentage,
            generator_lr = gen_lr,
            discriminator_lr = dis_lr,
            beta1 = beta1,
            beta2 = beta2,
            )

In [6]:
model_minerva.dis.state_dict()

OrderedDict([('backbone.0.cls_token',
              tensor([[[-1.8273,  0.5054,  0.4779, -0.5859, -0.8772, -0.3553,  0.0757,
                         0.6849,  1.6939, -1.1195,  2.1592, -1.2616,  0.1732, -2.2021,
                         2.0738,  0.9102, -1.3696, -0.3719, -0.1047, -0.4587,  0.0747,
                        -0.1992, -1.0800,  1.2603,  1.5449, -0.0343,  1.6790, -0.7909,
                        -1.4164, -0.0479, -0.4308,  1.0491,  1.6145,  0.5897, -0.9877,
                         0.6719, -1.6488, -0.0389, -1.3345, -1.6784,  1.0482,  0.7479,
                         0.8765, -0.4572, -0.2323, -0.5042, -0.2932, -0.7336,  0.9796,
                        -1.5475]]])),
             ('backbone.0.positions',
              tensor([[ 0.1759, -0.4979, -1.2948, -1.0442, -0.1158, -0.4223,  0.7558, -0.1147,
                        0.0695, -0.6511, -0.2151,  1.0565, -0.7020, -0.7504,  0.3539,  0.0453,
                        0.1445, -0.0448,  1.4053, -0.6119, -1.5403,  1.4804, -0.3550, -

In [7]:
model_original.dis.state_dict()

OrderedDict([('0.cls_token',
              tensor([[[-1.8273,  0.5054,  0.4779, -0.5859, -0.8772, -0.3553,  0.0757,
                         0.6849,  1.6939, -1.1195,  2.1592, -1.2616,  0.1732, -2.2021,
                         2.0738,  0.9102, -1.3696, -0.3719, -0.1047, -0.4587,  0.0747,
                        -0.1992, -1.0800,  1.2603,  1.5449, -0.0343,  1.6790, -0.7909,
                        -1.4164, -0.0479, -0.4308,  1.0491,  1.6145,  0.5897, -0.9877,
                         0.6719, -1.6488, -0.0389, -1.3345, -1.6784,  1.0482,  0.7479,
                         0.8765, -0.4572, -0.2323, -0.5042, -0.2932, -0.7336,  0.9796,
                        -1.5475]]])),
             ('0.positions',
              tensor([[ 0.1759, -0.4979, -1.2948, -1.0442, -0.1158, -0.4223,  0.7558, -0.1147,
                        0.0695, -0.6511, -0.2151,  1.0565, -0.7020, -0.7504,  0.3539,  0.0453,
                        0.1445, -0.0448,  1.4053, -0.6119, -1.5403,  1.4804, -0.3550, -0.4192,
          

Os MODELOS são parecidos, pelomenos a olho nu

In [8]:
set_seed()
tensor_1 = torch.rand(64, 100)
tensor_2 = torch.rand(64, 6, 60)

tensor_1.shape, tensor_2.shape

(torch.Size([64, 100]), torch.Size([64, 6, 60]))

In [9]:
set_seed()
a = model_minerva.gen(tensor_1)
set_seed()
b = model_original.gen(tensor_1)

(a == b).squeeze().float().mean().bool().item()

True

In [10]:
set_seed()
d = model_original.dis(tensor_2)
set_seed()
e = model_minerva.dis(tensor_2)

(d == e).float().mean().bool().item()

True

Saidas são idênticas com a mesma seed, logo não há diferenças práticas nos modelos, agora vamos ver nos treinamentos

In [11]:
device_number = 0
if torch.cuda.is_available():
    torch.cuda.set_device(device_number)

device = (
    f"cuda:{device_number}"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
display(f"Using {device} device")

'Using cuda:0 device'

In [12]:
torch.cuda.current_device()

0

In [13]:
newpath = r'../../../standardize_view'

In [14]:
dataNames = os.listdir(newpath)
dataNames.sort()
dataNames

['KuHar', 'MotionSense', 'RealWorld_thigh', 'RealWorld_waist', 'UCI', 'WISDM']

In [15]:
X = []
y = []   
for dataName in dataNames:
    print(dataName)
    dfTr = pd.read_csv(newpath + '/' + dataName + '/train.csv')
    X_tr = dfTr.values[:,:360].reshape(-1,6,60)
    y_tr = dfTr.values[:,-1].astype(np.int32)
    X.append(X_tr)
    y.append(y_tr)
X = np.concatenate(X, axis=0)
y = np.concatenate(y, axis=0)
X.shape, y.shape

KuHar
MotionSense
RealWorld_thigh
RealWorld_waist
UCI
WISDM


((36788, 6, 60), (36788,))

In [16]:
set_seed()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = True, random_state = 42)

X_train = torch.tensor(X_train.astype(np.float32), dtype=torch.float32, device=device).detach()
X_test = torch.tensor(X_test.astype(np.float32), dtype=torch.float32, device=device).detach()
y_train = torch.tensor(y_train.astype(np.float32), dtype=torch.float32, device=device).detach()
y_test = torch.tensor(y_test.astype(np.float32), dtype=torch.float32, device=device).detach()

X_train.shape, X_test.shape, y_train.shape, y_test.shape 

(torch.Size([29430, 6, 60]),
 torch.Size([7358, 6, 60]),
 torch.Size([29430]),
 torch.Size([7358]))

In [17]:
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, random_split

class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

class CustomDataModule(L.LightningDataModule):
    def __init__(self, data, labels, batch_size=16, val_split=0.2, num_workers=8):
        super().__init__()
        self.data = data
        self.labels = labels
        self.batch_size = batch_size
        self.val_split = val_split
        self.num_workers = num_workers

    def setup(self, stage=None):
        dataset = CustomDataset(self.data, self.labels)
        val_size = int(len(dataset) * self.val_split)
        train_size = len(dataset) - val_size
        self.train_dataset, self.val_dataset = random_split(dataset, [train_size, val_size])

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=self.num_workers)

# Supondo que X seja seu dado com forma (36788, 6, 60)
#X = np.random.rand(36788, 6, 60).astype(np.float32)  # Certifique-se de que os dados sejam do tipo float32
#y = np.random.randint(0, 2, size=(36788,)).astype(np.float32)  # Rótulos fictícios para fins de exemplo

X = X.astype(np.float32)
y = y.astype(int)
X_tensor = torch.tensor(X)
y_tensor = torch.tensor(y)

set_seed()
# Crie o DataModule
data_module = CustomDataModule(X_tensor, y_tensor, batch_size=64)

# Configure o DataModule
data_module.setup()

# Acesse os dataloaders
train_loader = data_module.train_dataloader()
val_loader = data_module.val_dataloader()

# Exemplo de iteração através do train_loader
print(len(train_loader))
print(len(val_loader))
for batch in train_loader:
    X_batch, y_batch = batch
    print(X_batch.shape, y_batch.shape)  # Deve imprimir torch.Size([64, 6, 60]) torch.Size([64])
    break

460
115
torch.Size([64, 6, 60]) torch.Size([64])


In [18]:
X_batch = X_batch.to('cuda:0')

In [19]:
from lightning.pytorch.callbacks import ModelCheckpoint
from callbacks import TsneGeneratorCallback, TsneEncoderCallback, KNNValidationCallback, MyPrintingCallback
from lightning.pytorch.callbacks import Callback

save_dir = './training'
name = 'ttsgan_error_hunt'

class SamplePrintingCallback(Callback):
    #Test callback just to do a start/end test for our training
    def __init__(self, test_batch = None, seed = 42):
        super().__init__()
        self.seed = seed
        self.batch = test_batch
        self.z = torch.tensor(np.random.normal(0, 1, (batch_size, 100)), dtype=torch.float, device='cuda')

    def on_train_epoch_end(self, trainer, pl_module):
        if self.batch is None:
            set_seed(self.seed)
            self.batch = next(iter(trainer.datamodule.val_dataloader()))
        

        set_seed(self.seed)
        #print(self.z.device, self.batch.device, pl_module.device)
        print()
        #print(self.z)
        #print(self.batch)
        print(pl_module.dis(self.batch))
        print(pl_module.gen(self.z))
        
printcallback = SamplePrintingCallback(test_batch=X_batch)

In [20]:
from lightning.pytorch.loggers.csv_logs import CSVLogger
version = 'minerva'
logger = CSVLogger(save_dir=save_dir, name=name, version=version)

In [21]:
if device.split(':')[0] == 'cuda':
    accelerator = 'cuda'

trainer = L.Trainer(accelerator=accelerator, devices=1,
                    callbacks=[printcallback], 
                    deterministic=True,
                    logger=logger, max_epochs=n_epochs) #max_steps=50000

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [22]:
set_seed()
trainer.fit(model = model_minerva, datamodule = data_module)

/usr/local/lib/python3.10/dist-packages/lightning/fabric/loggers/csv_logs.py:268: Experiment logs directory ./training/ttsgan_error_hunt/minerva exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory ./training/ttsgan_error_hunt/minerva/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]



  | Name     | Type                 | Params | Mode 
----------------------------------------------------------
0 | gen      | TTSGAN_Generator     | 65.3 K | train
1 | dis      | TTSGAN_Discriminator | 97.0 K | train
2 | loss_gen | MSELoss              | 0      | train
3 | loss_dis | MSELoss              | 0      | train
----------------------------------------------------------
162 K     Trainable params
0         Non-trainable params
162 K     Total params
0.649     Total estimated model params size (MB)
139       Modules in train mode
0         Modules in eval mode


Epoch 0: 100%|██████████| 460/460 [00:38<00:00, 11.86it/s, v_num=erva]     
tensor([[0.8877],
        [1.0460],
        [0.7173],
        [1.2805],
        [0.9052],
        [0.5703],
        [0.3915],
        [0.4610],
        [0.7342],
        [0.3074],
        [1.1011],
        [0.8002],
        [0.3547],
        [0.3305],
        [0.9731],
        [0.4941],
        [0.3740],
        [0.4119],
        [0.6142],
        [0.4482],
        [0.7448],
        [0.8907],
        [1.0959],
        [0.3029],
        [0.2919],
        [0.7220],
        [0.3369],
        [0.2627],
        [0.7488],
        [0.3807],
        [0.3624],
        [1.0615],
        [0.8348],
        [0.3400],
        [0.2973],
        [0.8574],
        [0.6493],
        [0.3376],
        [0.2849],
        [1.0366],
        [0.3999],
        [0.4533],
        [0.2345],
        [0.3654],
        [0.4373],
        [0.6439],
        [0.5783],
        [0.8959],
        [0.2822],
        [0.6583],
        [0.3715],
      

`Trainer.fit` stopped: `max_epochs=15` reached.


Epoch 14: 100%|██████████| 460/460 [00:39<00:00, 11.61it/s, v_num=erva]


In [31]:
version = 'original'
logger = CSVLogger(save_dir=save_dir, name=name, version=version)

In [32]:
trainer = L.Trainer(accelerator=accelerator, devices=1,
                    callbacks=[printcallback], 
                    deterministic=True,
                    logger=logger, max_epochs=n_epochs) #max_steps=50000

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [33]:
set_seed()
trainer.fit(model = model_original, datamodule = data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name     | Type          | Params | Mode 
---------------------------------------------------
0 | gen      | Generator     | 65.3 K | train
1 | dis      | Discriminator | 97.0 K | train
2 | loss_gen | MSELoss       | 0      | train
3 | loss_dis | MSELoss       | 0      | train
---------------------------------------------------
162 K     Trainable params
0         Non-trainable params
162 K     Total params
0.649     Total estimated model params size (MB)
138       Modules in train mode
0         Modules in eval mode


Epoch 0: 100%|██████████| 460/460 [00:40<00:00, 11.49it/s, v_num=inal]     
tensor([[0.8877],
        [1.0460],
        [0.7173],
        [1.2805],
        [0.9052],
        [0.5703],
        [0.3915],
        [0.4610],
        [0.7342],
        [0.3074],
        [1.1011],
        [0.8002],
        [0.3547],
        [0.3305],
        [0.9731],
        [0.4941],
        [0.3740],
        [0.4119],
        [0.6142],
        [0.4482],
        [0.7448],
        [0.8907],
        [1.0959],
        [0.3029],
        [0.2919],
        [0.7220],
        [0.3369],
        [0.2627],
        [0.7488],
        [0.3807],
        [0.3624],
        [1.0615],
        [0.8348],
        [0.3400],
        [0.2973],
        [0.8574],
        [0.6493],
        [0.3376],
        [0.2849],
        [1.0366],
        [0.3999],
        [0.4533],
        [0.2345],
        [0.3654],
        [0.4373],
        [0.6439],
        [0.5783],
        [0.8959],
        [0.2822],
        [0.6583],
        [0.3715],
      

`Trainer.fit` stopped: `max_epochs=15` reached.


Epoch 14: 100%|██████████| 460/460 [01:27<00:00,  5.25it/s, v_num=inal]


In [37]:
model_original.state_dict()

OrderedDict([('gen.pos_embed',
              tensor([[[ 2.8787e-02,  4.3386e-02,  3.7956e-02, -7.3199e-02,  7.1977e-03,
                         9.1581e-03, -1.2668e-02,  1.3456e-02,  6.1509e-03, -3.4398e-02],
                       [-2.8930e-02, -6.0553e-03,  2.9174e-02, -3.8373e-02,  4.2801e-02,
                         2.4416e-02, -1.1375e-02,  1.2327e-04,  2.5570e-03,  1.2023e-02],
                       [ 3.1600e-03,  6.3118e-02,  1.5847e-02, -1.8414e-02,  1.4429e-02,
                         2.4089e-02,  2.0032e-02, -4.4508e-02, -3.1956e-02, -3.7626e-02],
                       [-1.6131e-02, -1.0037e-02, -4.7325e-02,  1.5230e-03,  4.8273e-02,
                         1.5129e-02,  3.0909e-03,  2.4372e-02,  1.8236e-02,  2.7923e-02],
                       [ 1.1295e-02, -1.3193e-02, -3.0527e-02, -5.3686e-03, -2.8811e-02,
                        -2.1438e-02, -2.8717e-02, -3.6531e-02,  8.1224e-03,  4.6560e-04],
                       [-3.3519e-02, -4.5225e-03, -3.1530e-02,  6.4737e-02

In [38]:
model_minerva.state_dict()

OrderedDict([('gen.pos_embed',
              tensor([[[ 2.8787e-02,  4.3386e-02,  3.7956e-02, -7.3199e-02,  7.1977e-03,
                         9.1581e-03, -1.2668e-02,  1.3456e-02,  6.1509e-03, -3.4398e-02],
                       [-2.8930e-02, -6.0553e-03,  2.9174e-02, -3.8373e-02,  4.2801e-02,
                         2.4416e-02, -1.1375e-02,  1.2327e-04,  2.5570e-03,  1.2023e-02],
                       [ 3.1600e-03,  6.3118e-02,  1.5847e-02, -1.8414e-02,  1.4429e-02,
                         2.4089e-02,  2.0032e-02, -4.4508e-02, -3.1956e-02, -3.7626e-02],
                       [-1.6131e-02, -1.0037e-02, -4.7325e-02,  1.5230e-03,  4.8273e-02,
                         1.5129e-02,  3.0909e-03,  2.4372e-02,  1.8236e-02,  2.7923e-02],
                       [ 1.1295e-02, -1.3193e-02, -3.0527e-02, -5.3686e-03, -2.8811e-02,
                        -2.1438e-02, -2.8717e-02, -3.6531e-02,  8.1224e-03,  4.6560e-04],
                       [-3.3519e-02, -4.5225e-03, -3.1530e-02,  6.4737e-02